# Produce IO Tables from OECD data

In this notebook we use the OECD's data on Input-Output (IO) tables published [here](https://www.oecd.org/sti/ind/inter-country-input-output-tables.htm) to generate Australia's IO. At the moment, its output is a 36x36 matrix of intermediate consumption by sectors, which is extended with three additional rows accounting for imports, value added and total production by sector. Hence, the final output's dimenssions are 39x36.

## 1. Preliminaries
Import packages, load data, etc.

In [1]:
# Imports and path
import pymrio
import pandas as pd
from pathlib import Path
oecd_storage = Path('/project_data/data_asset/')

In [2]:
# Select the year to parse - 2015 in this case
oecd_path_year = pymrio.parse_oecd(path=oecd_storage, year=2015)

## 2. Extract domestic intermmediate consumption

We begin by extracting the symmetric input-ouput matrix of intermmediate consumption by sectors for one country. The matrix is called _Z_ in pymrio. We use Australia for the proof of concept.

In [3]:
# Extrac Z - The matrix has a multiindex at the column level [region, sector]. We slice by region. 
IO_AUS = oecd_path_year.Z.iloc[oecd_path_year.Z.index.get_level_values('region') == 'AUS', 
                               oecd_path_year.Z.columns.get_level_values('region') == 'AUS']
IO_AUS

region                  AUS                                         \
sector                01T03        05T06        07T08           09   
region sector                                                        
AUS    01T03   12764.521581   335.621861   134.544149     5.367220   
       05T06     138.811567  4195.535273  1014.166710   664.997136   
       07T08      87.176441   538.886717  5535.038857    75.254832   
       09         19.237516  5273.441198  2086.251593  1682.828181   
       10T12    5420.514236    46.981487    56.080848     5.898351   
       13T15      59.266539    40.950816    45.701526     2.277478   
       16         72.445542   188.352301   134.924692     8.241974   
       17T18     130.383801    89.022727    97.083161    11.355822   
       19        530.652825   313.953723   516.543989    64.253670   
       20T21     662.144018   220.207993   248.013542    22.264433   
       22        144.909099   222.825150   207.740458    24.270075   
       23         77.082897    58.593047   141.953793    15.743796   
       24         33.291006    86.599470   302.592939    45.808006   
       25        189.891075   401.866372   323.334077    65.518592   
       26         29.955417    36.942903    43.947246     7.682398   
       27         36.930932    53.620393    52.422516     7.311430   
       28        120.643314   238.652852   246.761811    46.966784   
       29         73.986464    51.506095    54.333544    12.916586   
       30         27.687685    50.811265     7.862453     0.866660   
       31T33      60.304468    59.203463    50.081345     8.166918   
       35T39    1374.454620  2584.481899  3854.070779   215.539340   
       41T43    1097.539410  1267.293094  3901.275352   226.086093   
       45T47    6859.781389  1838.619437  2565.375223   289.037127   
       49T53    1858.730874  2267.384782  2269.118950   387.108315   
       55T56     241.012593   208.905900   126.746452   117.731995   
       58T60      23.431683    32.800286    27.887134     2.037044   
       61        150.584150   168.482186   226.448017    36.752899   
       62T63      68.806194   442.281960   149.609418    23.816317   
       64T66    1958.584802  1646.774200  1695.828134   217.458605   
       68        226.021379   180.167605    73.453201    16.484610   
       69T82    1575.886286  3177.510842  1897.522638   565.613928   
       84         29.086759    78.608029    48.430456     6.634590   
       85         18.571376   156.844652    89.934544     3.031326   
       86T88     316.475985     8.568992     9.461411     0.992621   
       90T96     489.762255   574.351506  2352.199057    78.594125   
       97T98       0.000000     0.000000     0.000000     0.000000   

region                                                             \
sector                10T12       13T15           16        17T18   
region sector                                                       
AUS    01T03   22300.672875  135.980122  1323.988633    49.924049   
       05T06      82.613866    2.951015    10.235265    55.148532   
       07T08      88.410335    5.014642    15.515045    24.653482   
       09         27.565778    1.581559     5.497529     7.674105   
       10T12    6937.844451   40.889192    32.784869    65.106437   
       13T15      51.250799  252.121236    33.466274    27.782605   
       16         43.545735    3.084450  1349.815006   101.276054   
       17T18     422.261827   26.192141    71.741215  1773.541284   
       19        121.805725    8.512166    35.120236    38.677779   
       20T21     185.723622   49.921674    84.515034   160.159208   
       22        305.947016   19.285866    52.803018    97.400422   
       23        109.799465    2.688700    57.060365    13.444140   
       24         49.913339    2.470135    18.549173    68.238881   
       25        267.250335   16.478968   131.178508    50.462801   
       26         36.962610    2.073524     6.359049    10.838317   
       27         48.844025

In [4]:
# Check matrix is symmetrical - not only it needs to be nxn, but also that columns and rows are the same
# and ordered in the same fashion
check = IO_AUS.index == IO_AUS.columns 
assert check.all() == True 
print("No news is good news! Assertion true. :)")

No news is good news! Assertion true. :)


In [5]:
# IO_AUS.reset_index(drop = True, inplace = True)
IO_AUS.reset_index(level = 0, drop = True, inplace = True)
IO_AUS.columns = IO_AUS.columns.droplevel()
IO_AUS

sector,01T03,05T06,07T08,09,10T12,13T15,16,17T18,19,20T21,...,61,62T63,64T66,68,69T82,84,85,86T88,90T96,97T98
sector,,,,,,,,,,,,,,,,,,,,,
01T03,12764.521581,335.621861,134.544149,5.367220,22300.672875,135.980122,1323.988633,49.924049,4.598057,309.388619,...,35.016162,35.167907,49.240450,186.470176,782.246057,452.811382,326.715572,224.315559,256.608388,0
05T06,138.811567,4195.535273,1014.166710,664.997136,82.613866,2.951015,10.235265,55.148532,3696.839741,254.988513,...,35.363722,12.105347,14.835873,469.460153,204.522624,125.915558,108.696443,88.679568,139.706962,0
07T08,87.176441,538.886717,5535.038857,75.254832,88.410335,5.014642,15.515045,24.653482,16.050274,192.790035,...,33.055192,30.543600,31.131445,208.910359,186.551804,89.187144,81.333047,54.703897,69.717536,0
09,19.237516,5273.441198,2086.251593,1682.828181,27.565778,1.581559,5.497529,7.674105,5.186643,8.312155,...,10.641247,13.946927,9.397067,42.627355,118.950068,26.275031,13.753929,11.581111,14.841447,0
10T12,5420.514236,46.981487,56.080848,5.898351,6937.844451,40.889192,32.784869,65.106437,24.917292,323.651126,...,23.396680,27.459606,41.981435,83.631506,411.810282,691.456999,535.150278,779.659391,978.797545,0
13T15,59.266539,40.950816,45.701526,2.277478,51.250799,252.121236,33.466274,27.782605,6.154291,44.489286,...,7.819893,6.707509,5.003766,10.529053,75.462884,89.182270,48.078483,74.105308,181.387260,0
16,72.445542,188.352301,134.924692,8.241974,43.545735,3.084450,1349.815006,101.276054,5.991086,22.558744,...,4.496303,4.838831,3.646868,146.716691,154.111558,38.533910,55.338365,19.414393,121.280930,0
17T18,130.383801,89.022727,97.083161,11.355822,422.261827,26.192141,71.741215,1773.541284,18.895188,145.431717,...,113.284263,177.477701,301.157952,93.918922,832.309814,329.365167,466.133569,200.172843,276.365614,0
19,530.652825,313.953723,516.543989,64.253670,121.805725,8.512166,35.120236,38.677779,296.013959,233.702849,...,51.725652,51.488008,109.912268,148.471849,407.345120,283.402781,148.109304,109.385686,162.452750,0


In [6]:
# Check matrix is symmetrical - not only it needs to be nxn, but also that columns and rows are the same
# and ordered in the same fashion
check = IO_AUS.index == IO_AUS.columns 
assert check.all() == True 
print("No news is good news! Assertion true. :)")

No news is good news! Assertion true. :)


## 3. Imports

Our dataset contains inputs coming from other sectors in other countries. These constitute imports. While it would be possible to study them separately, for the sake of simplicity we will collapse them into a general category "imports". In the future, it could be interesting to explore international economic networks in more detail.

By including imports in our rows, we will be able to calculate a country's output at the column level (after adding value added and taxes). This doesn't mean that we can calculate output at the row level yet.

In [7]:
# Extrac all entries that are from non-Australian sectors feeding into Australian columns
IMP_AUS = oecd_path_year.Z.iloc[oecd_path_year.Z.index.get_level_values('region') != 'AUS',     # Non-Australian rows
                               oecd_path_year.Z.columns.get_level_values('region') == 'AUS']    # Australian columns
IMP_AUS

region               AUS                                                  \
sector             01T03     05T06     07T08            09         10T12   
region sector                                                              
ARG    01T03    3.782163  0.038990  0.015120  9.089364e-04  5.493334e+00   
       05T06    0.002691  0.000496  0.000321  7.288740e-05  3.161138e-03   
       07T08    0.000701  0.038134  0.811216  5.855111e-04  1.525414e-04   
       09       0.000000  0.165090  0.065091  5.294858e-02  0.000000e+00   
       10T12   29.549883  0.035620  0.020171  1.205696e-02  3.534676e+01   
...                  ...       ...       ...           ...           ...   
ZAF    84       0.000009  0.000100  0.000068  1.712856e-09  1.163744e-06   
       85       0.000548  0.000932  0.000436  1.867109e-04  2.979260e-04   
       86T88    0.000156  0.000268  0.000127  5.119082e-05  8.696424e-05   
       90T96    0.000032  0.000003  0.000008  5.931931e-06  2.247840e-07   
       97T98    0.000000  0.000000  0.000000  0.000000e+00  0.000000e+00   

region                                                                       \
sector                13T15            16         17T18        19     20T21   
region sector                                                                 
ARG    01T03   2.663934e-02  5.708350e-03  2.357739e-02  0.003944  0.156544   
       05T06   1.651705e-04  7.782361e-05  1.027439e-04  0.000027  0.000354   
       07T08   7.096227e-09  7.972431e-09  2.769896e-08  0.000084  0.018166   
       09      0.000000e+00  0.000000e+00  0.000000e+00  0.000000  0.000000   
       10T12   1.226207e-01  7.886244e-02  2.483554e-01  0.053753  1.618555   
...                     ...           ...           ...       ...       ...   
ZAF    84      2.528763e-06  2.841001e-06  9.870611e-06  0.000005  0.000038   
       85      1.634166e-05  6.865100e-05  1.659779e-04  0.000041  0.000440   
       86T88   5.406778e-06  2.074088e-05  5.004491e-05  0.000013  0.000126   
       90T96   1.500532e-06  2.941393e-05  1.083133e-04  0.000000  0.000000   
       97T98   0.000000e+00  0.000000e+00  0.000000e+00  0.000000  0.000000   

region         ...                                                      \
sector         ...            61         62T63         64T66        68   
region sector  ...                                                       
ARG    01T03   ...  9.080937e-05  6.987653e-05  8.742429e-04  0.018074   
       05T06   ...  2.421299e-04  3.972187e-04  2.609067e-04  0.000157   
       07T08   ...  2.766360e-08  1.968060e-07  1.790798e-07  0.001349   
       09      ...  0.000000e+00  0.000000e+00  0.000000e+00  0.000000   
       10T12   ...  2.193592e-03  1.885850e-03  9.009486e-03  0.220807   
...            ...           ...           ...           ...       ...   
ZAF    84      ...  9.858011e-06  7.013243e-05  6.381565e-05  0.000002   
       85      ...  6.388452e-04  9.669147e-04  6.059644e-04  0.000397   
       86T88   ...  2.058404e-04  2.951217e-04  2.230021e-04  0.000235   
       90T96   ...  7.897928e-04  2.806093e-04  9.289644e-04  0.000318   
       97T98   ...  0.000000e+00  0.000000e+00  0.000000e+00  0.000000   

region                                                                     
sector            69T82            84        85     86T88     90T96 97T98  
region sector                                                              
ARG    01T03   0.182714  2.718466e-01  0.229354  0.295526  0.179198     0  
       05T06   0.002148  7.160813e-04  0.000521  0.000604  0.000767     0  
       07T08   0.002027  3.833712e-07  0.000065  0.000014  0.000136     0  
       09      0.002508  0.000000e+00  0.000000  0.000000  0.000000     0  
       10T12   1.462561  3.059276e+00  2.650224  3.729207  2.305303     0  
...                 ...           ...       ...       ...       ...   ...  
ZAF    84      0.000585  1.366155e-04  0.000071  0.000044  0.000021     0  
       85      0.004313  5.0924

In [8]:
# Create a new dataframe that combines intermediate consumption (IO_AUS) with total imports, called IO_AUS_EXT
# The table is no longer symmetric
IO_AUS_EXT = IO_AUS.copy()
IO_AUS_EXT.loc['IMPORTS'] = pd.Series(IMP_AUS.sum()).values
IO_AUS_EXT

sector,01T03,05T06,07T08,09,10T12,13T15,16,17T18,19,20T21,...,61,62T63,64T66,68,69T82,84,85,86T88,90T96,97T98
sector,,,,,,,,,,,,,,,,,,,,,
01T03,12764.521581,335.621861,134.544149,5.367220,22300.672875,135.980122,1323.988633,49.924049,4.598057,309.388619,...,35.016162,35.167907,49.240450,186.470176,782.246057,452.811382,326.715572,224.315559,256.608388,0.0
05T06,138.811567,4195.535273,1014.166710,664.997136,82.613866,2.951015,10.235265,55.148532,3696.839741,254.988513,...,35.363722,12.105347,14.835873,469.460153,204.522624,125.915558,108.696443,88.679568,139.706962,0.0
07T08,87.176441,538.886717,5535.038857,75.254832,88.410335,5.014642,15.515045,24.653482,16.050274,192.790035,...,33.055192,30.543600,31.131445,208.910359,186.551804,89.187144,81.333047,54.703897,69.717536,0.0
09,19.237516,5273.441198,2086.251593,1682.828181,27.565778,1.581559,5.497529,7.674105,5.186643,8.312155,...,10.641247,13.946927,9.397067,42.627355,118.950068,26.275031,13.753929,11.581111,14.841447,0.0
10T12,5420.514236,46.981487,56.080848,5.898351,6937.844451,40.889192,32.784869,65.106437,24.917292,323.651126,...,23.396680,27.459606,41.981435,83.631506,411.810282,691.456999,535.150278,779.659391,978.797545,0.0
13T15,59.266539,40.950816,45.701526,2.277478,51.250799,252.121236,33.466274,27.782605,6.154291,44.489286,...,7.819893,6.707509,5.003766,10.529053,75.462884,89.182270,48.078483,74.105308,181.387260,0.0
16,72.445542,188.352301,134.924692,8.241974,43.545735,3.084450,1349.815006,101.276054,5.991086,22.558744,...,4.496303,4.838831,3.646868,146.716691,154.111558,38.533910,55.338365,19.414393,121.280930,0.0
17T18,130.383801,89.022727,97.083161,11.355822,422.261827,26.192141,71.741215,1773.541284,18.895188,145.431717,...,113.284263,177.477701,301.157952,93.918922,832.309814,329.365167,466.133569,200.172843,276.365614,0.0
19,530.652825,313.953723,516.543989,64.253670,121.805725,8.512166,35.120236,38.677779,296.013959,233.702849,...,51.725652,51.488008,109.912268,148.471849,407.345120,283.402781,148.109304,109.385686,162.452750,0.0


## 4. Taxes and Value Added

Finally, we include two extra rows to account for taxes/subsidies and value added in each industry. These are contained in pymrio's _F_ matrix.

### 4.1. Taxes and subsidies

In [9]:
# Break down of taxes and subsidies and value added by sector
oecd_path_year.factor_inputs.F

region               ARG                                          \
sector             01T03         05T06        07T08           09   
inputtype                                                          
AUS_TAXSUB      0.132996      0.017576     0.016490     0.008126   
AUT_TAXSUB      0.052867      0.026761     0.015243     0.004556   
BEL_TAXSUB      0.225642      0.048434     0.036728     0.009054   
CAN_TAXSUB      0.122170      0.033575     0.054038     0.007309   
CHL_TAXSUB      0.315159      0.110207     0.086002     0.015152   
...                  ...           ...          ...          ...   
THA_TAXSUB      0.032990      0.027859     0.014391     0.004627   
TUN_TAXSUB      0.000728      0.000132     0.000095     0.000018   
VNM_TAXSUB     -0.006366      0.005721     0.004746     0.000962   
ROW_TAXSUB      0.239255      5.829891     0.118004     0.704568   
VALU        53012.211324  18153.567067  7977.452693  2339.346224   

region                                                                         \
sector             10T12         13T15           16        17T18           19   
inputtype                                                                       
AUS_TAXSUB      0.006253      0.082651     0.008937     0.036073     0.061325   
AUT_TAXSUB      0.053804      0.037633     0.010132     0.029677     0.009379   
BEL_TAXSUB      0.094257      0.106845     0.012598     0.047739     0.083867   
CAN_TAXSUB      0.064599      0.060188     0.008411     0.030119     0.042757   
CHL_TAXSUB      0.364894      0.839357     0.047639     0.190712     0.100263   
...                  ...           ...          ...          ...          ...   
THA_TAXSUB      0.032936      0.035228     0.006278     0.011148     0.007591   
TUN_TAXSUB      0.000055      0.000588     0.000056     0.000209     0.000168   
VNM_TAXSUB     -0.033038      0.685171     0.003647     0.004392     0.001354   
ROW_TAXSUB      0.393384      0.634194     0.030984     0.275608    40.399715   
VALU        28804.517746  13604.234042  3260.350050  6174.541292  9701.303023   

region                    ...          ZAF                            \
sector             20T21  ...           61       62T63         64T66   
inputtype                 ...                                          
AUS_TAXSUB      0.199232  ...     0.132380    0.040280      0.035792   
AUT_TAXSUB      0.060642  ...     0.096526    0.047824      0.008710   
BEL_TAXSUB      0.245115  ...     0.085472    0.133854      0.050437   
CAN_TAXSUB      0.151584  ...     0.058905    0.015413      0.003724   
CHL_TAXSUB      0.360592  ...     0.004691    0.012607      0.003957   
...                  ...  ...          ...         ...           ...   
THA_TAXSUB      0.028013  ...     0.083390    0.035060      0.008409   
TUN_TAXSUB      0.001169  ...     0.000486    0.000279      0.000065   
VNM_TAXSUB      0.009083  ...     0.110833    0.010020      0.004147   
ROW_TAXSUB      2.015715  ...    -0.091858    0.293590      0.003026   
VALU        10816.660864  ...  5009.412934  736.075802  27315.023870   

region                                                             \
sector                68         69T82            84           85   
inputtype                                                           
AUS_TAXSUB      0.068539      0.219342      0.269904     0.079234   
AUT_TAXSUB      0.060626      0.161540      0.371547     0.048500   
BEL_TAXSUB      0.143190      0.395236      0.579650     0.117868   
CAN_TAXSUB      0.017423      0.051784      0.080184     0.016149   
CHL_TAXSUB      0.035575      0.054481      0.062479     0.042086   
...                  ...           ...           ...          ...   
THA_TAXSUB      0.041220      0.115086      0.292701     0.079553   
TUN_TAXSUB      0.000523      0.001237      0.001362     0.000581   
VNM_TAXSUB      0.012975      0.066512      0.185915     0.080701   
ROW_TAXSUB      0.285604      0.706826      2.824502     0.354555   
VALU  

First, we take the taxes and subsidies paid in Australia.

In [10]:
# Break down of taxes and subsidies by sector just for Australia
AUS_TAXSUB = oecd_path_year.factor_inputs.F.iloc[oecd_path_year.factor_inputs.F.index.get_level_values('inputtype') == 'AUS_TAXSUB',
                                                   oecd_path_year.factor_inputs.F.columns.get_level_values('region') == 'AUS']
AUS_TAXSUB.columns = AUS_TAXSUB.columns.droplevel()
AUS_TAXSUB

sector,01T03,05T06,07T08,09,10T12,13T15,16,17T18,19,20T21,...,61,62T63,64T66,68,69T82,84,85,86T88,90T96,97T98
inputtype,,,,,,,,,,,,,,,,,,,,,
AUS_TAXSUB,952.03943,1072.901373,1309.449877,197.938503,2826.686253,153.634766,126.634534,204.805311,628.397424,474.32364,...,266.508189,234.868465,497.677048,918.23948,1413.587964,1264.248504,580.238743,657.600697,962.311856,0


Australian firms have also paid taxes or received subsidies outside their national borders, so we need to account for that.

In [11]:
AUS_INT_TAXSUB = oecd_path_year.factor_inputs.F.iloc[oecd_path_year.factor_inputs.F.index.get_level_values('inputtype') != 'AUS_TAXSUB', # select all non-Australian rows 
                                                     oecd_path_year.factor_inputs.F.columns.get_level_values('region') == 'AUS']         # select only Australian columns
AUS_INT_TAXSUB.columns = AUS_INT_TAXSUB.columns.droplevel()
AUS_INT_TAXSUB = AUS_INT_TAXSUB[AUS_INT_TAXSUB.index != 'VALU'] # Drop value added row
AUS_INT_TAXSUB

sector,01T03,05T06,07T08,09,10T12,13T15,16,17T18,19,20T21,...,61,62T63,64T66,68,69T82,84,85,86T88,90T96,97T98
inputtype,,,,,,,,,,,,,,,,,,,,,
AUT_TAXSUB,0.345302,0.217607,0.171735,0.033690,0.448357,0.044266,0.050967,0.095736,0.012431,0.135693,...,0.083340,0.050281,0.040670,0.076220,0.324831,0.677607,0.121368,0.448356,0.340850,0
BEL_TAXSUB,0.568421,0.167085,0.212792,0.032509,0.524488,0.059280,0.038294,0.114094,0.088062,0.345239,...,0.032798,0.038739,0.059055,0.084929,0.360566,0.248794,0.129034,0.769611,0.183534,0
CAN_TAXSUB,0.261932,0.178023,0.223113,0.026664,0.239284,0.023039,0.056673,0.051232,0.020948,0.139912,...,0.083513,0.042327,0.050304,0.080333,0.247572,0.370282,0.078487,0.149631,0.149340,0
CHL_TAXSUB,0.097740,0.035132,0.249971,0.003411,0.115108,0.004240,0.063796,0.041446,0.003400,0.032198,...,0.007785,0.008375,0.013564,0.013228,0.069663,0.039907,0.025542,0.025399,0.056596,0
CZE_TAXSUB,0.069763,0.043229,0.040680,0.008324,0.077178,0.019661,0.024719,0.028822,0.002313,0.019438,...,0.022333,0.013676,0.011255,0.026440,0.125808,0.071386,0.033323,0.042584,0.067770,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TWN_TAXSUB,3.173019,1.940218,3.163254,0.400849,0.492151,0.065339,0.192869,0.234215,1.802569,1.432919,...,0.311792,0.292062,0.658304,0.904241,2.480383,1.794307,0.905843,0.664921,0.988605,0
THA_TAXSUB,1.746008,1.190275,1.573012,0.257889,0.655799,0.080707,0.185625,0.192226,0.646235,0.661016,...,0.280893,0.188388,0.333006,0.611040,2.590759,1.546829,0.592932,0.517597,0.966718,0
TUN_TAXSUB,-0.000989,0.000231,0.000255,0.000052,-0.002277,0.000366,0.000085,0.000053,0.000013,0.000071,...,0.000161,0.000059,0.000056,0.000146,0.000584,0.000513,0.000243,0.000553,0.000777,0


### 4.2. Value added

In [12]:
# Value added of Australian sectors
AUS_VA = oecd_path_year.factor_inputs.F.iloc[oecd_path_year.factor_inputs.F.index.get_level_values('inputtype') == 'VALU',
                                                   oecd_path_year.factor_inputs.F.columns.get_level_values('region') == 'AUS']
AUS_VA.columns = AUS_VA.columns.droplevel()
AUS_VA

sector,01T03,05T06,07T08,09,10T12,13T15,16,17T18,19,20T21,...,61,62T63,64T66,68,69T82,84,85,86T88,90T96,97T98
inputtype,,,,,,,,,,,,,,,,,,,,,
VALU,29967.551571,30822.642351,36063.880794,5662.469088,18863.258018,1807.502925,3184.117359,4069.667497,3541.457641,7165.378425,...,16688.014055,24171.358469,106210.29824,139120.521033,105995.596906,67700.753809,60082.199558,83994.282022,31935.523313,0


### 4.3. Add extra rows to extended IO table

Two alternatives. Comment out one.

In [13]:
# ALTERNATIVE 1: ADD TAXES AND VALUE ADDED SEPARATELY. THE VALUE ADDED ROW HERE WOULD ACCOUNT FOR TAXES AS WELL
# THIS WOULDN'T AFFECT THE TOTAL, AS TAXES ARE MULTIPLIED BY -1
# IO_AUS_EXT = IO_AUS_EXT.append(AUS_TAXSUB * (-1))  # Multiply taxes by -1. We need to subtract them from Value Added row, otherwise they would be accounted for twice
# IO_AUS_EXT.loc['AUS_INT_TAXSUB'] = pd.Series(AUS_INT_TAXSUB.sum()).values * (-1) # See line above
# IO_AUS_EXT = IO_AUS_EXT.append(AUS_VA)
# IO_AUS_EXT

In [14]:
# ALTERNATIVE 2: SUBTRACT TAXES FROM VALUE ADDED AND DON'T INCLUDE THEM IN FINAL 
VA_TAXES = AUS_VA.append(AUS_TAXSUB * (-1))
VA_TAXES.loc['AUS_INT_TAXSUB'] = pd.Series(AUS_INT_TAXSUB.sum()).values * (-1)

IO_AUS_EXT.loc['VALUE ADDED'] = pd.Series(VA_TAXES.sum()).values
IO_AUS_EXT

sector,01T03,05T06,07T08,09,10T12,13T15,16,17T18,19,20T21,...,61,62T63,64T66,68,69T82,84,85,86T88,90T96,97T98
sector,,,,,,,,,,,,,,,,,,,,,
01T03,12764.521581,335.621861,134.544149,5.367220,22300.672875,135.980122,1323.988633,49.924049,4.598057,309.388619,...,35.016162,35.167907,49.240450,186.470176,782.246057,452.811382,326.715572,224.315559,256.608388,0.0
05T06,138.811567,4195.535273,1014.166710,664.997136,82.613866,2.951015,10.235265,55.148532,3696.839741,254.988513,...,35.363722,12.105347,14.835873,469.460153,204.522624,125.915558,108.696443,88.679568,139.706962,0.0
07T08,87.176441,538.886717,5535.038857,75.254832,88.410335,5.014642,15.515045,24.653482,16.050274,192.790035,...,33.055192,30.543600,31.131445,208.910359,186.551804,89.187144,81.333047,54.703897,69.717536,0.0
09,19.237516,5273.441198,2086.251593,1682.828181,27.565778,1.581559,5.497529,7.674105,5.186643,8.312155,...,10.641247,13.946927,9.397067,42.627355,118.950068,26.275031,13.753929,11.581111,14.841447,0.0
10T12,5420.514236,46.981487,56.080848,5.898351,6937.844451,40.889192,32.784869,65.106437,24.917292,323.651126,...,23.396680,27.459606,41.981435,83.631506,411.810282,691.456999,535.150278,779.659391,978.797545,0.0
13T15,59.266539,40.950816,45.701526,2.277478,51.250799,252.121236,33.466274,27.782605,6.154291,44.489286,...,7.819893,6.707509,5.003766,10.529053,75.462884,89.182270,48.078483,74.105308,181.387260,0.0
16,72.445542,188.352301,134.924692,8.241974,43.545735,3.084450,1349.815006,101.276054,5.991086,22.558744,...,4.496303,4.838831,3.646868,146.716691,154.111558,38.533910,55.338365,19.414393,121.280930,0.0
17T18,130.383801,89.022727,97.083161,11.355822,422.261827,26.192141,71.741215,1773.541284,18.895188,145.431717,...,113.284263,177.477701,301.157952,93.918922,832.309814,329.365167,466.133569,200.172843,276.365614,0.0
19,530.652825,313.953723,516.543989,64.253670,121.805725,8.512166,35.120236,38.677779,296.013959,233.702849,...,51.725652,51.488008,109.912268,148.471849,407.345120,283.402781,148.109304,109.385686,162.452750,0.0


## 5. Total production

Now we have all the components that make up an industry's total output. We just need to add them up.

In [15]:
IO_AUS_EXT.loc['TOTAL PRODUCTION'] = pd.Series(IO_AUS_EXT.sum()).values
IO_AUS_EXT

sector,01T03,05T06,07T08,09,10T12,13T15,16,17T18,19,20T21,...,61,62T63,64T66,68,69T82,84,85,86T88,90T96,97T98
sector,,,,,,,,,,,,,,,,,,,,,
01T03,12764.521581,335.621861,134.544149,5.367220,22300.672875,135.980122,1323.988633,49.924049,4.598057,309.388619,...,35.016162,35.167907,49.240450,186.470176,782.246057,452.811382,326.715572,224.315559,256.608388,0.0
05T06,138.811567,4195.535273,1014.166710,664.997136,82.613866,2.951015,10.235265,55.148532,3696.839741,254.988513,...,35.363722,12.105347,14.835873,469.460153,204.522624,125.915558,108.696443,88.679568,139.706962,0.0
07T08,87.176441,538.886717,5535.038857,75.254832,88.410335,5.014642,15.515045,24.653482,16.050274,192.790035,...,33.055192,30.543600,31.131445,208.910359,186.551804,89.187144,81.333047,54.703897,69.717536,0.0
09,19.237516,5273.441198,2086.251593,1682.828181,27.565778,1.581559,5.497529,7.674105,5.186643,8.312155,...,10.641247,13.946927,9.397067,42.627355,118.950068,26.275031,13.753929,11.581111,14.841447,0.0
10T12,5420.514236,46.981487,56.080848,5.898351,6937.844451,40.889192,32.784869,65.106437,24.917292,323.651126,...,23.396680,27.459606,41.981435,83.631506,411.810282,691.456999,535.150278,779.659391,978.797545,0.0
13T15,59.266539,40.950816,45.701526,2.277478,51.250799,252.121236,33.466274,27.782605,6.154291,44.489286,...,7.819893,6.707509,5.003766,10.529053,75.462884,89.182270,48.078483,74.105308,181.387260,0.0
16,72.445542,188.352301,134.924692,8.241974,43.545735,3.084450,1349.815006,101.276054,5.991086,22.558744,...,4.496303,4.838831,3.646868,146.716691,154.111558,38.533910,55.338365,19.414393,121.280930,0.0
17T18,130.383801,89.022727,97.083161,11.355822,422.261827,26.192141,71.741215,1773.541284,18.895188,145.431717,...,113.284263,177.477701,301.157952,93.918922,832.309814,329.365167,466.133569,200.172843,276.365614,0.0
19,530.652825,313.953723,516.543989,64.253670,121.805725,8.512166,35.120236,38.677779,296.013959,233.702849,...,51.725652,51.488008,109.912268,148.471849,407.345120,283.402781,148.109304,109.385686,162.452750,0.0


## 6. Export data

In [16]:
IO_AUS_EXT.to_csv('/project_data/data_asset/IO_AUS.csv')